In [49]:
from Distribution import *

class RV:
    def __init__(self, distr:"Distribution"):
        self.distribution = distr

    def realize(self):
        return self.distribution.dist.rvs()

    

In [50]:
norm = Normal(0, 1)

In [51]:
x = RV(distr=Normal(0, 1))

In [52]:
x.realize()

-0.6439974421110816

In [53]:
import numpy as np
import math
from RV import *
from Sample import *
import scipy.stats as sci

class Distribution:
    
    def plot_pdf(self):
        pass

    def plot_cdf(self):
        pass


class Normal(Distribution):
    def __init__(self, mean:float, var:float):
        self.mean = mean
        self.var = var
        self.std = np.sqrt(self.var) #Add Handling negative variance
        self.dist = sci.norm(loc=self.mean, scale=self.std)
        self.support = (-np.inf, np.inf)

    def __str__(self):
        return f"N(mean={self.mean}, variance={self.var})"
    
    def pdf(self, x):
        return self.dist.pdf(x)
    
    def cdf(self, x):
        return self.dist.cdf(x)
    
    def mgf(self, t):
        return self.dist.mgf(t)
    
    def draw(self, n=1):
        if n==1:
            return RV(distr=self)
        else:
            return Sample([RV(distr=self) for i in range(n)])


class Uniform(Distribution):
    def __init__(self, a, b):
        self.a, self.b = a, b
        self.E = (self.a+self.b)/2
        self.dist = sci.uniform(loc=a, scale=b-a)
    
    def pdf(self, x):
        return self.dist.pdf(x)
    
    def cdf(self, x):
        return self.dist.cdf(x)
    

class Binomial(Distribution):
    def __init__(self, n:int, p:float):
        self.n = n
        self.p = p
        self.dist = sci.binom(n=self.n, p=self.p)

    def pmf(self, x):
        return self.dist.pmf(x)
    
    def cdf(self, x):
        return self.dist.cdf(x)
    

class Poisson(Distribution):
    def __init__(self, mu):
        pass

    def pmf(self, x):
        return self.dist.pmf(x)
    
    def cdf(self, x):
        return self.dist.cdf(x)


class Exponential(Distribution):
    def __init__(self, intensity: float):
        self.intensity = intensity
        self.dist = sci.expon(scale=1/intensity)

    def mgf(self, t):
        return self.intensity / (self.intensity - t)


class Gamma(Distribution):
    def __init__(self, theta, r):
        pass

Distr = Distribution
N = Normal

if __name__ == "__main__":
    X = RV(distr=Binomial(n=20, p=0.1))
    print(X.distribution)


In [54]:
import numpy as np
from RV import *

class Sample:
    def __init__(self, rvs:list[RV]):
        self.rvs = rvs

    def observe(self):
        return np.array([rv.realize() for rv in self.rvs])
    
    def __str__(self):
        return 


In [55]:
normal_dist = Normal(0, 1)
s1 = normal_dist.draw(40)


In [56]:
a = RV(Normal(0, 1))
a.realize()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/anaconda3/envs/dexter/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/28/kdd3c0m500l1pjc158c5hgww0000gn/T/ipykernel_3772/2152302921.py", line 2, in <module>
    a.realize()
  File "/Users/deniz/Desktop/Programming/Packages/dexter/Stat/RV.py", line 8, in realize
    return self.distribution.dist.rvs()
           ^^^^^^^^^^
AttributeError: 'RV' object has no attribute 'distr'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/envs/dexter/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/dexter/lib/python3.12/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
  

In [57]:
print(s1.observe())

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/anaconda3/envs/dexter/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/28/kdd3c0m500l1pjc158c5hgww0000gn/T/ipykernel_3772/1747391732.py", line 1, in <module>
    print(s1.observe())
          ^^^^^^^^^^^^
  File "/var/folders/28/kdd3c0m500l1pjc158c5hgww0000gn/T/ipykernel_3772/4215420137.py", line 9, in observe
    return np.array([rv.realize() for rv in self.rvs])
                     ^^^^^^^^^^^^
  File "/Users/deniz/Desktop/Programming/Packages/dexter/Stat/RV.py", line 8, in realize
    return self.distribution.dist.rvs()
           ^^^^^^^^^^
AttributeError: 'RV' object has no attribute 'distr'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/envs/dexter/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
  